# Connectome analysis
## Set parameters

In [ ]:
import os

CONFIG_FILE_NAME = "braian_config.toml"                     # assumes the file is in DATA_ROOT directory
# EXPERIMENT_DIRECTORY = "p4"
# EXPERIMENT_DIRECTORY = "experiment"
# EXPERIMENT_DIRECTORY = "proof"
EXPERIMENT_DIRECTORY = "rebased_on_mjd"

USE_REMOTE_DATA = False                                     # if True, it tries to read the data on the laboratory's server
# ###################################### REMOTE DIRECTORIES #####################################
IS_COLLABORATION_PROJ = False
COLLABORATION_DIRECTORY = os.path.join("Mathias Schmidt", "soumnya")

# ###################################### LOCAL DIRECTORIES ######################################
# DATA_ROOT  = f"../data/experiments/sowmya/{EXPERIMENT_DIRECTORY}"
# PLOTS_ROOT = f"../plots/sowmya/{EXPERIMENT_DIRECTORY}"
DATA_ROOT  = f"../data/experiments/{EXPERIMENT_DIRECTORY}"
PLOTS_ROOT = f"../plots/{EXPERIMENT_DIRECTORY}"

In [ ]:
# on which comparison of CONFIG_FILE_NAME to run the connectome analysis
COMPARISON_ID = 2

In [ ]:
# ####################################### GENERAL OPTIONS #######################################
SAVED_PLOT_EXTENSION = ".html"                              # '.html' for interactive plot
                                                            # '.svg' for vectorized image
                                                            # '.png'/'.jpg'/... for rasterized image

# ###################################### CORRELATION MATRIX #####################################
MATRIX_SAVE_PLOT = False
MATRIX_SHOW_PLOT = False
MATRIX_CELL_HEIGHT = 18
MATRIX_STAR_SIZE = 8
MATRIX_CELL_RATIO = 1
MATRIX_MIN_PLOT_HEIGHT = 500

# ######################################## CHORD DIAGRAM ########################################
CHORD_SAVE_PLOT = False
CHORD_SHOW_PLOT = True
CHORD_PLOT_SIZE = 1200
CHORD_NO_BACKGROUND = False
CHORD_PLOT_ISOLATED_REGIONS = True
CHORD_REGIONS_SIZE = 10
CHORD_REGIONS_FONT_SIZE = 10
CHORD_MAX_EDGE_WIDTH = 3
CHORD_USE_WEIGHTED_EDGE_WIDTHS = False
CHORD_USE_COLORSCALE_EDGES = True
CHORD_BOTTOM_ANNOTATIONS = dict(
    annotation1 = "Dark grey nodes are regions with insufficient data to compute cross correlation",
    annotation2 = "Light grey nodes are regions with no correlation with others above the threshold",
    annotation3 = "This is the third annotation",
    # howmany annotations desired with the following format:
    # annotations<k> = "<annotation>"
)

In [ ]:
import igraph as ig

# https://doi.org/10.1109/TKDE.2007.190689
# clustering_args = (ig.Graph.community_optimal_modularity,)
                    # Graphs with up to fifty vertices should be fine, graphs with a couple of hundred vertices might be possible.
                    # crashes on my pc with summary structure's Graph
clustering_args = (ig.Graph.community_fastgreedy,)
# clustering_args = (ig.Graph.community_infomap,)
# clustering_args = (ig.Graph.community_leading_eigenvector_naive,)
# clustering_args = (ig.Graph.community_leading_eigenvector,)
# clustering_args = (ig.Graph.community_label_propagation,)
# clustering_args = (ig.Graph.community_multilevel,)
# clustering_args = (ig.Graph.community_edge_betweenness, dict(directed=False))
# clustering_args = (ig.Graph.community_spinglass,)
# clustering_args = (ig.Graph.community_walktrap,)
# clustering_args = (ig.Graph.community_leiden,)

In [ ]:
# layout_fun = ig.Graph.layout_kamada_kawai               # 1st best - good for ~small networks# 
layout_fun = ig.Graph.layout_fruchterman_reingold       # 2nd best - good for bigger networks
# layout_fun = ig.Graph.layout_graphopt                   # 3rd best
# layout_fun = ig.Graph.layout_davidson_harel             # 4th best
# layout_fun = ig.Graph.layout_mds                        # 5th best

## Script's code
run all cell below

In [ ]:
import os
import sys
import random
from plotly.colors import DEFAULT_PLOTLY_COLORS

project_path = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(project_path)
import BraiAn

In [ ]:
if USE_REMOTE_DATA:
    DATA_ROOT, PLOTS_ROOT = BraiAn.remote_dirs(EXPERIMENT_DIRECTORY, IS_COLLABORATION_PROJ, COLLABORATION_DIRECTORY)

config_file = os.path.join(DATA_ROOT, CONFIG_FILE_NAME)

In [ ]:
import tomllib

with open(config_file, "rb") as f:
    config = tomllib.load(f)
config
# ######################################### LOAD CONFIG #########################################
EXPERIMENT_NAME = config["experiment"]["name"]

ATLAS_VERSION = config["atlas"]["version"]
BRANCHES_TO_EXCLUDE = config["atlas"]["excluded-branches"]
USE_LITERATURE_REUNIENS =  config["atlas"]["use-literature-reuniens"]       # add a 'REtot' region, merging the following regions: 'RE', 'Xi', 'RH'

NORMALIZATION = config["brains"]["normalization"]                           # call get_normalization_methods() on a AnimalGroup object to know its available normalization methods
MIN_AREA = config["connectome"]["min-area"]                                 # area in mm². If a region of one animal is smaller, that same animal won't be displayed in the plots
REGIONS_TO_PLOT_SELECTION_METHOD = config["connectome"]["regions-to-plot"]  # Available options are:
                                                                            #   - "summary structures"
                                                                            #   - "nre to bla"
                                                                            #   - "major divisions"
                                                                            #   - "depth <n>" where <n> is an integer of the depth desired
                                                                            #   - "structural level <n>" where <n> is an integer of the level desired
                                                                            #   - "pls <experiment> <salience_threshold>" (e.g., "pls proof 1.2")
                                                                            # where <n> is an integer of the depth/level desired
GROUPED_REGIONS = config["connectome"]["grouped-regions"]

# ###################################### CROSS CORRELATION ######################################
MIN_ANIMALS_CROSS_CORRELATION = config["connectome"]["min-animals"]         # 'max'/None means that ALL the animals must have the region, otherwise the correlation is NaN
if MIN_ANIMALS_CROSS_CORRELATION == "max":
    MIN_ANIMALS_CROSS_CORRELATION = None
ONLY_REGIONS_PRESENT_IN_ALL_GROUPS = config["connectome"]["only-regions-in-all-compared-groups"]# if False, the correlation matrix and the chord plots of two groups may refer to different regions
ONLY_REGIONS_WITH_SUFFICIENT_DATA = config["connectome"]["only-regions-with-sufficient-data"]   # what to do with brain regions with less animals than MIN_ANIMALS_CROSS_CORRELATION

# ######################################### CONNECTOME ##########################################
FC_WEIGHTED = config["connectome"]["functional"]["weighted"]
PC_WEIGHTED = config["connectome"]["pruned"]["weighted"]
SC_WEIGHTED = config["connectome"]["structural"]["weighted"]
FC_USE_NEGATIVE_LINKS = config["connectome"]["functional"]["use-negative-links"]
PC_USE_NEGATIVE_LINKS = config["connectome"]["pruned"]["use-negative-links"]
FC_P_CUTOFF = config["connectome"]["functional"]["p-cutoff"]
FC_R_CUTOFF = config["connectome"]["functional"]["r-cutoff"]
SC_LOG10_CUTOFF = config["connectome"]["structural"]["log10-cutoff"]

if FC_USE_NEGATIVE_LINKS:
    # see https://plotly.com/python/builtin-colorscales/
    COLORSCALE = "RdBu_r"
    CHORD_COLORSCALE_MIN = -1
else:
    COLORSCALE = "Magma"
    CHORD_COLORSCALE_MIN = "cutoff"

from collections import namedtuple
GroupDirectory = namedtuple("GroupDirectory", "id name dirs")
groups = [
    GroupDirectory(
        id=int(group[len("group"):])-1,
        name=config["experiment"][group]["name"],
        dirs=config["experiment"][group]["dirs"]
    ) for group in config["experiment"] if group.startswith("group") and group[len("group"):].isdigit()
]

comparison_groups = config["comparison"][str(COMPARISON_ID)]["groups"]
Comparison = namedtuple("Comparison", "groups dir")
comparison = Comparison(
    groups=[groups[id-1] for id in comparison_groups],
    dir=config["comparison"][str(COMPARISON_ID)]["dir"]
)

In [ ]:
data_input_path     = os.path.join(DATA_ROOT, "BraiAn_output")
data_output_path    = os.path.join(data_input_path, comparison.dir)
plots_output_path   = os.path.join(PLOTS_ROOT, comparison.dir)

if not(os.path.exists(data_output_path)):
    os.makedirs(data_output_path, exist_ok=True)

if not(os.path.exists(plots_output_path)):
    os.makedirs(plots_output_path, exist_ok=True)

In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
path_to_allen_json = os.path.join(project_path, "data", "AllenMouseBrainOntology.json")
BraiAn.cache(path_to_allen_json, "http://api.brain-map.org/api/v2/structure_graph_download/1.json")
AllenBrain = BraiAn.AllenBrainHierarchy(path_to_allen_json, BRANCHES_TO_EXCLUDE, use_literature_reuniens=USE_LITERATURE_REUNIENS, version=ATLAS_VERSION)

In [ ]:
animal_groups: list[BraiAn.AnimalGroup] = []
for group in comparison.groups:
    animal_group = BraiAn.AnimalGroup.from_csv(group.name, data_input_path, f"cell_counts_{group.name}.csv")
    animal_groups.append(animal_group)
    print(f"Group '{animal_group.name}' - #animals: {animal_group.n}, marker: {animal_group.marker}")


In [ ]:
match REGIONS_TO_PLOT_SELECTION_METHOD:
    case "summary structures":
        # selects the Summary Strucutures
        path_to_summary_structures = os.path.join(project_path, "data", "AllenSummaryStructures.csv")
        AllenBrain.select_from_csv(path_to_summary_structures, include_nre_tot=USE_LITERATURE_REUNIENS)
    case "nre to bla":
        # selects the NRe to BLA inputs
        path_to_inputs = os.path.join(project_path, "data", "NRe_to_BLA_inputs.csv")
        AllenBrain.select_from_csv(path_to_inputs, include_nre_tot=USE_LITERATURE_REUNIENS)
        nre_bla_regions = AllenBrain.get_selected_regions()
        nre_bla_regions += ("REtot", "BLA")
        AllenBrain.select_regions(nre_bla_regions)
    case "major divisions":
        AllenBrain.select_regions(BraiAn.MAJOR_DIVISIONS)
    case s if s.startswith("pls"):
        options = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")
        assert len(options) == 3, "The 'REGIONS_TO_PLOT_SELECTION_METHOD' option is invalid. Make sure it follows the follows the following pattern: \"pls <experiment> <salience_threshold>\" (e.g., \"pls proof 1.2\")"
        pls_experiment, pls_threshold = options[1:]
        pls_threshold = pls_threshold.replace(".", "_")
        assert len(animal_groups) == 2, f"You can't use the PLS of '{pls_experiment}' for selecting the regions to plot because '{comparison.dir}' has too many groups ({len(animal_groups)})"
        pls_file = f"pls_{animal_groups[0].marker}_{NORMALIZATION}_salient_regions_above_{pls_threshold}.csv".lower()
        regions_to_plot_pls_csv = os.path.abspath(os.path.join(DATA_ROOT, os.pardir, pls_experiment, "BraiAn_output", comparison.dir, pls_file))
        assert os.path.isfile(regions_to_plot_pls_csv), f"Could not find the file '{regions_to_plot_pls_csv}'"
        AllenBrain.select_from_csv(regions_to_plot_pls_csv, key="acronym")
    case s if s.startswith("depth"):
        n = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")[-1]
        try:
            depth = int(n)
        except Exception:
            raise Exception("Could not retrieve the <n> parameter of the 'depth' method for 'REGIONS_TO_PLOT_SELECTION_METHOD'")
        AllenBrain.select_at_depth(depth)
    case s if s.startswith("structural level"):
        n = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")[-1]
        try:
            level = int(n)
        except Exception:
            raise Exception("Could not retrieve the <n> parameter of the 'structural level' method for 'REGIONS_TO_PLOT_SELECTION_METHOD'")
        AllenBrain.select_at_structural_level(level)
    case _:
        raise Exception(f"Invalid value '{REGIONS_TO_PLOT_SELECTION_METHOD}' for REGIONS_TO_PLOT_SELECTION_METHOD")

if len(GROUPED_REGIONS) > 0:
    AllenBrain.add_to_selection(GROUPED_REGIONS)
regions_to_plot = AllenBrain.get_selected_regions()
print(f"You selected {len(regions_to_plot)} regions to plot.")

In [ ]:
for animal_group in animal_groups:
    animal_group.remove_smaller_subregions(MIN_AREA, regions_to_plot, AllenBrain)

In [ ]:
if MATRIX_SAVE_PLOT or MATRIX_SHOW_PLOT or CHORD_SAVE_PLOT or CHORD_SHOW_PLOT:
    groups_cross_correlations = [BraiAn.CrossCorrelation(g, regions_to_plot, AllenBrain, NORMALIZATION, MIN_ANIMALS_CROSS_CORRELATION, g.name) for g in animal_groups]
    
    if ONLY_REGIONS_PRESENT_IN_ALL_GROUPS:
        BraiAn.CrossCorrelation.make_comparable(*groups_cross_correlations)
    if ONLY_REGIONS_WITH_SUFFICIENT_DATA:
        for cc in groups_cross_correlations:
            cc.remove_insufficient_regions()
    regions_to_plot_selection_method_str = REGIONS_TO_PLOT_SELECTION_METHOD.replace(".", "_").replace(" ", "_")

In [ ]:
if MATRIX_SAVE_PLOT or MATRIX_SHOW_PLOT:
    for group, cc in zip(animal_groups, groups_cross_correlations):
        title = f"{group.name} Pearson cross correlation matrix (n = {group.n})"
        fig = cc.plot(
                title=title,
                cell_height=MATRIX_CELL_HEIGHT, min_plot_height=MATRIX_MIN_PLOT_HEIGHT,
                star_size=MATRIX_STAR_SIZE, aspect_ratio=MATRIX_CELL_RATIO,
                # color_min=-1, color_max=1,
                color_min=0, color_max=1, colorscale=COLORSCALE
                )
        if MATRIX_SAVE_PLOT:
            plot_filename = f"correlation_matrix_min{MIN_ANIMALS_CROSS_CORRELATION}_{group.name}_{group.marker}_{NORMALIZATION}_{regions_to_plot_selection_method_str}{SAVED_PLOT_EXTENSION}".lower()
            plot_filepath = os.path.join(plots_output_path, plot_filename)
            match SAVED_PLOT_EXTENSION.lower():
                case ".html":
                    fig.write_html(plot_filepath, config=dict(toImageButtonOptions=dict(format="svg")))
                case _:
                    fig.write_image(plot_filepath)
        if MATRIX_SHOW_PLOT:
            fig.show()

In [ ]:
FCs = []
for animal_group, cc in zip(animal_groups, groups_cross_correlations):
    connectome = BraiAn.FunctionalConnectome(cc,
                                         p_cutoff=FC_P_CUTOFF, r_cutoff=FC_R_CUTOFF,
                                         negatives=FC_USE_NEGATIVE_LINKS, weighted=FC_WEIGHTED)
    FCs.append(connectome)

In [ ]:
p_str = str(FC_P_CUTOFF).replace(".", "_")
r_str = str(FC_R_CUTOFF).replace(".", "_")
for fc in FCs:
    # if REGIONS_TO_PLOT_SELECTION_METHOD == "nre to bla" and "REtot" in cc.p.index:
    #     connectome = connectome.region_subgraph("REtot", isolated_vertices=True)
    title = f"{fc.name} connectomics graph from Pearson correlation (n = {fc.n}, {'|r|' if FC_USE_NEGATIVE_LINKS else 'r'} >= {fc.r_cutoff}, p <= {fc.p_cutoff})"
    fig = BraiAn.draw_chord_plot(fc,
                                AllenBrain=AllenBrain,
                                ideograms_arc_index=50,
                                title=title,
                                size=CHORD_PLOT_SIZE,
                                no_background=CHORD_NO_BACKGROUND,
                                isolated_regions=CHORD_PLOT_ISOLATED_REGIONS,
                                regions_size=CHORD_REGIONS_SIZE,
                                regions_font_size=CHORD_REGIONS_FONT_SIZE,
                                max_edge_width=CHORD_MAX_EDGE_WIDTH,
                                use_weighted_edge_widths=CHORD_USE_WEIGHTED_EDGE_WIDTHS,
                                colorscale_edges=CHORD_USE_COLORSCALE_EDGES,
                                colorscale=COLORSCALE,
                                colorscale_min=CHORD_COLORSCALE_MIN,
    )
    fig.show()
    # filename = f"chord_plot_min{MIN_ANIMALS_CROSS_CORRELATION}_p{p_str}_r{r_str}_{animal_group.name.lower()}_{animal_groups[0].marker}_{NORMALIZATION}_{regions_to_plot_selection_method_str}.html"
    # fig.write_html(filename.lower(), config=dict(toImageButtonOptions=dict(format="svg")))

In [ ]:
#for animal_group, cc in zip(animal_groups, groups_cross_correlations):
for fc in FCs:
    #if REGIONS_TO_PLOT_SELECTION_METHOD == "nre to bla" and "REtot" in cc.p.index:
    #    connectome = connectome.region_subgraph("REtot", isolated_vertices=False)
    fc.cluster_regions(*clustering_args)
    fc.participation_coefficient(weights=False) # doesn't work on weighted connectomics
    
    title = f"{fc.name} connectomics graph from Pearson correlation (n = {fc.n}, {'|r|' if FC_USE_NEGATIVE_LINKS else 'r'} >= {fc.r_cutoff}, p <= {fc.p_cutoff})"
    random.seed(0) # used by layout_fun to arrange the nodes of the connectome
    fig = BraiAn.draw_network_plot(fc, layout_fun, AllenBrain, title=title,
                                   isolated_regions=False,
                                   use_centrality=True, centrality_metric="Participation coefficient", width=1250)
    fig.show()
    # filename = f"network_min{MIN_ANIMALS_CROSS_CORRELATION}_p{p_str}_r{r_str}_{animal_group.name.lower()}_{animal_groups[0].marker}_{NORMALIZATION}_{regions_to_plot_selection_method_str}.html"
    # fig.write_html(filename.lower(), config=dict(toImageButtonOptions=dict(format="svg")))

In [ ]:
import numpy as np
import pandas as pd
# from https://download.alleninstitute.org/publications/A_high_resolution_data-driven_model_of_the_mouse_connectome/
normalized_connection_density_file = os.path.join(project_path, "data",
                                                    "A_high_resolution_data-driven_model_of_the_mouse_connectome",
                                                    "normalized_connection_density.csv")
BraiAn.cache(normalized_connection_density_file,
             "https://download.alleninstitute.org/publications/A_high_resolution_data-driven_model_of_the_mouse_connectome/normalized_connection_density.csv")

In [ ]:
allen_connectome = BraiAn.StructuralConnectome(normalized_connection_density_file, regions_to_plot, AllenBrain, mode="max", log10_cutoff=SC_LOG10_CUTOFF, weighted=SC_WEIGHTED, isolated_vertices=False)
print(f"""
Max: {allen_connectome.A.max()}
Mean: {allen_connectome.A.mean()}+-{allen_connectome.A.std()}
Mean (log10): {allen_connectome.A.mean(log=True)}+-{allen_connectome.A.std(log=True)}
Density: {allen_connectome.G.density()}
""")

In [ ]:
# same as Figure 2 in https://direct.mit.edu/netn/article/3/1/217/2194/High-resolution-data-driven-model-of-the-mouse
fig = allen_connectome.plot_adjacency(color_min=-5, color_max=-2.5, cell_height=4)
fig.show()

In [ ]:
re = allen_connectome.G.vs.select(name="RE")[0]
print("RE->BLA density:", allen_connectome.A.data.loc["RE", "BLA"], f"(log10={np.log10(allen_connectome.A.data.loc['RE', 'BLA'])})")
print("BLA->RE density:", allen_connectome.A.data.loc["BLA", "RE"], f"(log10={np.log10(allen_connectome.A.data.loc['BLA', 'RE'])})")
print("RE->ILA density:", allen_connectome.A.data.loc["RE", "ILA"], f"(log10={np.log10(allen_connectome.A.data.loc['RE', 'ILA'])})")
print("ILA->RE density:", allen_connectome.A.data.loc["ILA", "RE"], f"(log10={np.log10(allen_connectome.A.data.loc['ILA', 'RE'])})")
print("BLA->Xi density:", allen_connectome.A.data.loc["BLA", "Xi"], f"(log10={np.log10(allen_connectome.A.data.loc['BLA', 'Xi'])})")
print("Xi->BLA density:", allen_connectome.A.data.loc["Xi", "BLA"], f"(log10={np.log10(allen_connectome.A.data.loc['Xi', 'BLA'])})")

print(f"\nNetwork with log10(density) cutoff={SC_LOG10_CUTOFF} ({10**SC_LOG10_CUTOFF})")
print("\tRE Degree:", re.degree(), f"({re.indegree()}+{re.outdegree()})")
# for e in re.all_edges():
#     print(f"\t\t{e.source_vertex['name']}->{e.target_vertex['name']}: {e['weight']:.3f} ({10**e['weight']:.6f})")

In [ ]:
if False:
    fig = BraiAn.draw_chord_plot(allen_connectome,
                                AllenBrain=AllenBrain,
                                ideograms_arc_index=50,
                                title=allen_connectome.name+f" [log10(normalized density) >= {SC_LOG10_CUTOFF}]",
                                size=CHORD_PLOT_SIZE,
                                no_background=CHORD_NO_BACKGROUND,
                                regions_size=CHORD_REGIONS_SIZE,
                                regions_font_size=CHORD_REGIONS_FONT_SIZE,
                                max_edge_width=CHORD_MAX_EDGE_WIDTH,
                                use_weighted_edge_widths=False, #CHORD_USE_WEIGHTED_EDGE_WIDTHS,
                                colorscale_edges=CHORD_USE_COLORSCALE_EDGES,
                                colorscale=COLORSCALE,
                                colorscale_min=10**SC_LOG10_CUTOFF,
                                colorscale_max=10**(-2.5) #np.log10(allen_connection_matrix.A.max(axis=None))
    )
    fig.show()

In [ ]:
FCs_pruned = []
for cc in groups_cross_correlations:
    fc_pruned = BraiAn.PrunedConnectomics(allen_connectome, cc, FC_P_CUTOFF, FC_R_CUTOFF, negatives=PC_USE_NEGATIVE_LINKS, weighted=PC_WEIGHTED, isolated_vertices=True)
    fc_pruned.G.vs["Degree"] = fc_pruned.G.degree(mode="all")
    fc_pruned.G.vs["In-degree"] = fc_pruned.G.degree(mode="in")
    fc_pruned.G.vs["Out-degree"] = fc_pruned.G.degree(mode="out")
    fc_pruned.G.vs["Strength"] = fc_pruned.G.strength(mode="all")
    fc_pruned.G.vs["Betweeness"] = fc_pruned.G.betweenness(directed=True)#, weights="weight")
    fc_pruned.G.vs["Closeness (out)"] = fc_pruned.G.closeness(mode="out", normalized=True)#, weights="weight")
    fc_pruned.G.vs["Harmonic (out)"] = fc_pruned.G.harmonic_centrality(mode="out", normalized=True)#, weights="weight")
    fc_pruned.G.vs["PageRank"] = fc_pruned.G.pagerank(directed=True)#, weights="weight")
    fc_pruned.G.vs["Eigenvector"] = fc_pruned.G.eigenvector_centrality(scale=True, directed=True, weights=None)
    fc_pruned.cluster_regions(ig.Graph.community_infomap)
    FCs_pruned.append(fc_pruned)

In [ ]:
# control group
for fc, fc_pruned in zip(FCs, FCs_pruned):
    print(fc.name, end=":\n")
    print("\t#nodes (functional):", fc.G.vcount())
    print("\t#nodes (pruned):", fc_pruned.G.vcount())
    print("\t#edges (functional):", fc.G.ecount()) # undirected --> multiply by 2
    print("\t#edges (pruned):", fc_pruned.G.ecount())
    print("\tavg pruning-survivor edges:", fc_pruned.G.ecount() / 2, f"({fc_pruned.G.ecount()/(fc.G.ecount()*2)*100:0.3f}%)")
    print("\tdensity (structural):", allen_connectome.G.density())
    print("\tdensity (functional):", fc.G.density())
    print("\tdensity (pruned):", fc_pruned.G.density())

In [ ]:
# are edges that where cut still reachable?

import plotly.graph_objects as go
for fc, fc_pruned in zip(FCs, FCs_pruned):
    ds_pruned = fc_pruned.get_functional_neighbors_distances()
    ds_unique, ds_counts = np.unique(ds_pruned, return_counts=True)
    fig = go.Figure(
            go.Pie(
                labels=[int(d) if not np.isinf(d) else "infinite" for d in ds_unique],
                values=ds_counts,
                marker=dict(
                    line=dict(color="#000000", width=2)
                ),
                hovertemplate = "<b>Pruned distance</b>: %{label}<br>"+
                                "<b>#functional connection</b>: %{value} (%{percent})<extra></extra>",
                sort=False,
                textfont=dict(size=12),
                hole=0.3,
                textinfo="label", textposition="outside",
                showlegend=True
            ))
    title = f"{fc_pruned.name}: minimum distance between regions that previously to pruning were functionally connected"
    fig.update_layout(title=title)
    fig.show()
    # reached_ds_pruned = ds_pruned[~np.isinf(ds_pruned)]
    # n_unreachable_es = len(ds_pruned) - len(reached_ds_pruned)
    # print(n_unreachable_es, len(reached_ds_pruned), np.mean(reached_ds_pruned), np.std(reached_ds_pruned))
    # # percentage of unreachable brain regions that were previously functionally connected
    # print(n_unreachable_es / fc.G.ecount() * 100)

In [ ]:
fc_pruned_A = BraiAn.ConnectomeAdjacency(fc_pruned.A, AllenBrain)
fig = fc_pruned_A.plot(cell_height=4, colorscale=COLORSCALE, color_max=1)
fig.update_layout(yaxis=dict(scaleanchor="x"))

In [ ]:
for fc_pruned in FCs_pruned:
    title = f"{fc_pruned.name} [n = {cc.n}, {'|r|' if PC_USE_NEGATIVE_LINKS else 'r'} >= {fc_pruned.r_cutoff}, p <= {fc_pruned.p_cutoff}, d >= {10**SC_LOG10_CUTOFF}]"
    random.seed(0) # used by layout_fun to arrange the nodes of the connectome
    fig = BraiAn.draw_network_plot(fc_pruned, layout_fun, AllenBrain, title=title, use_centrality=True, centrality_metric="Harmonic (out)", width=1000, isolated_regions=False)
    # fig = BraiAn.draw_network_plot(fc_pruned, layout_fun, AllenBrain, title=title, use_centrality=False, use_clustering=False, width=1000, isolated_regions=False)
    # fig = BraiAn.draw_chord_plot(fc_pruned,
    #                             AllenBrain=AllenBrain,
    #                             ideograms_arc_index=50,
    #                             title=title,
    #                             size=CHORD_PLOT_SIZE,
    #                             no_background=CHORD_NO_BACKGROUND,
    #                             isolated_regions=False,
    #                             regions_size=CHORD_REGIONS_SIZE,
    #                             regions_font_size=CHORD_REGIONS_FONT_SIZE,
    #                             max_edge_width=CHORD_MAX_EDGE_WIDTH,
    #                             use_weighted_edge_widths=CHORD_USE_WEIGHTED_EDGE_WIDTHS,
    #                             colorscale_edges=CHORD_USE_COLORSCALE_EDGES,
    #                             colorscale=CHORD_COLORSCALE,
    #                             colorscale_min=NETWORK_R_CUTOFF, #CHORD_COLORSCALE_MIN,
    # )
    fig.show()
    # fig.write_html(f"chord_{fc_pruned.name.lower()}.html")
    # fig.write_html(f"{fc_pruned.name.lower()}.html") #_r{NETWORK_R_CUTOFF}_p{NETWORK_P_CUTOFF}_d{log10_cutoff}.html")

In [ ]:
re = fc_pruned.G.vs.select(name="VPL")[0]
for e in re.all_edges():
    d = e["weight"] # fc_pruned. .data.loc[e.source_vertex['name'], e.target_vertex['name']]
    p = cc.p.data.loc[e.source_vertex['name'], e.target_vertex['name']]
    print(d >= 10**SC_LOG10_CUTOFF, 10**SC_LOG10_CUTOFF, SC_LOG10_CUTOFF)
    print(f"{e.source_vertex['name']}->{e.target_vertex['name']}: r={e['weight']:.3f} p={p:.4f} d={d:.5f} ({np.log10(d):.6f})")

In [ ]:
import igraph as ig
import plotly.graph_objects as go
import numpy as np

def importance_score(scores, ranks):
    return 1/np.sqrt(ranks+1)
    #return 1/(ranks+1)

def normalize(x: np.array, nmin=0):
    x = x-nmin #np.nanmin(x)
    return x/np.nanmax(x)

def normalize_centrality(vs: ig.VertexSeq, centrality: str):
    if centrality not in vs.attributes():
        raise ValueError(f"Invalid attribute name for centrality '{centrality}'")
    return normalize(np.asarray(vs[centrality]))

def get_ranks(vs: ig.VertexSeq, centrality: str):
    if centrality not in vs.attributes():
        raise ValueError(f"Invalid attribute name for centrality '{centrality}'")
    centrality_scores = np.nan_to_num(vs[centrality], copy=False, nan=0.0)
    i_sorted_centrality = np.flip(np.argsort(centrality_scores))
    centrality_scores = centrality_scores[i_sorted_centrality]
    _, unique_ranks = np.unique(centrality_scores, return_index=True) # returns indices of the unique centrality scores (in ascending order)
    ranks_repetitions = unique_ranks[:-1]-unique_ranks[1:]
    ranks_repetitions = np.insert(ranks_repetitions, 0, len(vs)-unique_ranks[0])
    ranks = np.repeat(unique_ranks, ranks_repetitions)
    return ranks[np.flip(i_sorted_centrality).argsort()]

def centrality_barplot(G: ig.Graph, centrality, group_by, offsetgroup, first_n, **kwargs):
    # group_by is, most probably, either 'upper_region', 'cluster', or 'name'
    if centrality not in G.vs.attributes():
        raise ValueError(f"Invalid attribute name for centrality '{centrality}'")
    if first_n is not None:
        vs_indices = sorted(G.vs.indices, key=lambda i: np.nan_to_num(G.vs[i][centrality], nan=0.0), reverse=True)[:first_n]
    else:
        vs_indices = G.vs.indices
    sorted_vs = sorted(vs_indices, key=lambda i: (G.vs[i][group_by], np.nan_to_num(G.vs[i][centrality], nan=0.0)), reverse=True)
    sorted_vs = ig.VertexSeq(G, sorted_vs)
    # normalization on max can only be done if
    scores = normalize_centrality(sorted_vs, centrality)
    ranks = get_ranks(sorted_vs, centrality)
    return go.Bar(
        x=sorted_vs[group_by],
        y=normalize(scores),
        hovertext=[
            f"Region: {v['name']}<br>"+
            f"{centrality}: {v[centrality]:.5f}<br>"+
            f"rank: {rank}"
            for v,rank in zip(sorted_vs, ranks)],
        name=centrality,
        offsetgroup=offsetgroup,
        **kwargs
    )

def plot_centralities(connectome: BraiAn.Connectome, *attrs: str, group_by="upper_region", first_n=None, **kwargs):
    barplots = []
    for i, centrality in enumerate(attrs):
        barplot = centrality_barplot(connectome.G, centrality, group_by=group_by, offsetgroup=i, first_n=first_n, **kwargs)
        barplots.append(barplot)
    return go.Figure(
        data=barplots,
        layout=dict(
            title=f"Regions ranks with different metrics (grouped by '{group_by}')",
            yaxis=dict(
                title="Normalized centrality score",
                side="left",
            ),
        ))

for fc_pruned in FCs_pruned:
    # "Degree", "In-degree", "Out-degree", "Strength" # these metrics appear to roughly say the same thing in all connectomes
    # fig = plot_centralities(fc_pruned, "Degree", "Betweeness", "Eigenvector", "PageRank", "Closeness (out)", group_by="upper_region") #, "Harmonic (out)")
    fig = plot_centralities(fc_pruned, "Degree", "Betweeness", "Eigenvector", "PageRank", "Harmonic (out)", group_by="name", first_n=20) #"upper_region") #, "Harmonic (out)")
    fig.show()

In [ ]:
A = np.asarray(
[[0,   1,   0.1, 0],
 [1,   0,   0,   1],
 [0.1, 0,   0,   0.1], 
 [0,   1,   0.1, 0]])
G = ig.Graph.Weighted_Adjacency(A, loops=False)
G.get_all_shortest_paths(0, to=3), G.get_shortest_paths(0, to=3, weights=1/np.asarray(G.es["weight"], dtype=float), output="vpath"), G.get_shortest_paths(0, to=3, weights="weight", output="vpath")

In [ ]:
#sp = connectome.G.get_shortest_paths(0, to=5, output="epath")

import functools
import operator
def random_walk_probability(G: ig.Graph, path, use_weights=True):
    if len(path) == 0:
        # the two nodes are unreachable
        return 0
    if type(path) != ig.EdgeSeq:
        path = G.es[path]
    if G.is_weighted() and use_weights:
        weights = functools.reduce(operator.mul, path["weight"], 1)
        source_strengths = functools.reduce(lambda res, e: res * e.source_vertex.strength(mode="out", loops=False, weights="weight"), path, 1)
    else:
        weights = 1
        source_strengths = functools.reduce(lambda res, e: res * e.source_vertex.outdegree(loops=False), path, 1)
    return weights / source_strengths

# test random_walk_probability
outdegree = 2
v_start = np.where(np.asarray(fc_pruned.G.outdegree(), dtype=int) == outdegree)[0][0]
v_end = fc_pruned.G.vs[v_start].neighbors()[0].index
remapped_weights = 1/np.asarray(fc_pruned.G.es["weight"], dtype=float)
# igraph intends weights as "cost" not as "strength"
sp = fc_pruned.G.get_shortest_paths(v_start, to=v_end, weights=remapped_weights, output="epath")[0]
random_walk_probability(fc_pruned.G, sp, use_weights=False), sp

In [ ]:
import itertools
for connectome in itertools.chain.from_iterable(zip(FCs, FCs_pruned)):
    go.Figure(
        [go.Histogram(
            x=connectome.G.degree(),
            xbins=dict(start=0, size=2,)
        )],
        layout=dict(
            # bargap=0.1,
            title=f"{connectome.name}: degree distribution"
        )
    #    layout=dict(bargap=0.1)
    ).show()

In [ ]:
import igraph as ig
import numpy as np

def path_length(G: ig.Graph, unreachable_nodes=True):
    if G.is_weighted():
        ds = np.asarray(G.distances(weights=np.abs(G.es["weight"])), dtype=float)
    else:
        ds = np.asarray(G.distances(), dtype=float)
    if unreachable_nodes:
        ds[ds == np.inf] = 0
        N = G.vcount()
        return ds.sum(axis=0) / (N-1)
    else:
        ds[ds == np.inf] = np.nan
        Ns = (~np.isnan(ds)).sum(axis=0, dtype=float)
        den = Ns - 1
        den[den == 0] = np.nan
        return np.nansum(ds, axis=0) / den

def average_path_length(G: ig.Graph, unreachable_nodes=True):
    # if unreachable_nodes=False, it's equal to igraph.Graph.average_path_length(unconn=True)
    if G.is_weighted():
        ds = np.asarray(G.distances(weights=np.abs(G.es["weight"]), mode="out"), dtype=float)
    else:
        ds = np.asarray(G.distances(), dtype=float)
    if unreachable_nodes:
        ds[ds == np.inf] = 0
        N = G.vcount()
        return ds.sum() / (N * (N - 1))
    else:
        ds = ds[ds != np.inf]
        return ds.sum() / (len(ds) - G.vcount())

# import networkx as nx

# Gnx = nx.Graph([])
# Gnx.add_node(1)
# nx.global_efficiency(Gnx) # <- returns 0
def nodal_efficiency(G: ig.Graph): # same as harmonic centrality (normalized and mode="out")
    # https://en.wikipedia.org/wiki/Efficiency_(network_science)
    N = G.vcount()
    if N == 0:
        raise ValueError("Empty graph")
    elif N == 1:
#        return np.full(N, 1, dtype=float)
        return np.full(N, 0, dtype=float)
    if G.is_weighted():
        # ws = np.abs(1.0 / np.asarray(G.es["weight"], dtype=float))
        ws = np.abs(np.asarray(G.es["weight"], dtype=float))
        ds = np.asarray(G.distances(mode="out", weights=ws), dtype=float)
    else:
        ds = np.asarray(G.distances(mode="out"), dtype=float)
    np.fill_diagonal(ds, np.NaN)
    efficiency = 1 / ds
    np.fill_diagonal(efficiency, 0) # we don't want to consider pathlength to itself
    ne = np.apply_along_axis(sum, 1, efficiency) / (N-1)
    return ne

def global_efficiency(G):
    if G.vcount() == 0:
        return np.nan
    # return np.asarray(G.harmonic_centrality(mode="out", normalized=True, weights="weight" if G.is_weighted() else None))
    return nodal_efficiency(G).mean()

def local_efficiency(G, zero_degree=True):
    local_efficiency_i = []
    for i in range(G.vcount()):
        G_i = G.induced_subgraph(G.neighbors(i), "create_from_scratch")
        global_efficiency_i = global_efficiency(G_i)
        local_efficiency_i.append(global_efficiency_i)
    if zero_degree:
        return np.nansum(np.asarray(local_efficiency_i)) / len(local_efficiency_i)
    else:
        return np.nanmean(np.asarray(local_efficiency_i))

In [ ]:
for connectome in itertools.chain.from_iterable(zip(FCs, FCs_pruned)):
    print(f"{connectome.name} global efficiency:", global_efficiency(connectome.G))

In [ ]:
def print_stats(G: ig.Graph):
    print(f"""
    INFO:
        graph type: {'Weighted' if G.is_weighted() else 'Not weighted'}
        #regions: {G.vcount()}
        #connected regions: {len([d for d in G.degree() if d > 0])}
        Max degree: {G.maxdegree()}
    SEGREGATION:
        Cluster coefficient: {G.transitivity_undirected()}
        Mean local cluster coefficient (d>=2): {np.nanmean(G.transitivity_local_undirected())}
        Mean local cluster coefficient (all): {np.nansum(G.transitivity_local_undirected()) / G.vcount()}
        Local efficiency (d>=1): {local_efficiency(G, zero_degree=False)}
        Local efficiency (all): {local_efficiency(G, zero_degree=True)}
    INTEGRATION_
        Global efficiency: {global_efficiency(G)}
        Avg path length (∞ -> 0): {path_length(G, unreachable_nodes=True).mean()}
        Avg path length (no ∞): {G.average_path_length(unconn=True)}
        Median [characteristic] path lengh (∞ -> 0): {np.nanmedian(path_length(G, unreachable_nodes=True))}
        Median [characteristic] path lengh (no ∞): {np.nanmedian(path_length(G, unreachable_nodes=False))}
    """)

print_stats(connectome.G)
print_stats(fc_pruned.G)

In [ ]:
# WARNING: certain measures of statistical dependence used to quantify functional connectivity can bias
# network organization in a way that cannot be removed by topological rewiring.
# 
G_rewired = FCs[1].G.copy()
G_rewired.rewire(mode="loops")      # functional connectomes are inherently more clustered and exaggerate features such as small-worldness.
                                    # We should use Maslov-Sneppen rewiring method (Bullmore, et al. 2016 - Fundamentals of Brain Network Analysis, chapter 10.3)
#G_rewired.rewire(mode="simple")    # does not create/destroy loop edges. If you allow it, you may change the degrees
G_rewired.es["weight"] = FCs[1].G.es["weight"]
print_stats(G_rewired)

In [ ]:
fc_collapsed = fc_pruned
for subregion in AllenBrain.direct_subregions["Isocortex"]:
    fc_collapsed = fc_collapsed.collapse_region(AllenBrain, subregion)

In [ ]:
fc_collapsed.G.vs["Degree"] = fc_collapsed.G.degree(mode="all")
fc_collapsed.G.vs["In-degree"] = fc_collapsed.G.degree(mode="in")
fc_collapsed.G.vs["Out-degree"] = fc_collapsed.G.degree(mode="out")
fc_collapsed.G.vs["Strength"] = fc_collapsed.G.strength(mode="all")
fc_collapsed.G.vs["Betweeness"] = fc_collapsed.G.betweenness(directed=True)#, weights="weight")
fc_collapsed.G.vs["Closeness (out)"] = fc_collapsed.G.closeness(mode="out", normalized=True)#, weights="weight")
fc_collapsed.G.vs["Harmonic (out)"] = fc_collapsed.G.harmonic_centrality(mode="out", normalized=True)#, weights="weight")
fc_collapsed.G.vs["PageRank"] = fc_collapsed.G.pagerank(directed=True)#, weights="weight")
fc_collapsed.G.vs["Eigenvector"] = fc_collapsed.G.eigenvector_centrality(scale=True, directed=True, weights=None)
fc_collapsed.cluster_regions(ig.Graph.community_infomap)
from BraiAn.connectome.utils_bu import participation_coefficient
fc_collapsed.G.vs["Participation coefficient"] = participation_coefficient(fc_collapsed.G, fc_collapsed.vc)

In [ ]:
title = f"{fc_collapsed.name}"
random.seed(0) # used by layout_fun to arrange the nodes of the connectome
fig = BraiAn.draw_network_plot(fc_collapsed, layout_fun, AllenBrain, title=title, use_centrality=True, centrality_metric="Harmonic (out)", width=1000, isolated_regions=False)
# fig = BraiAn.draw_network_plot(fc_collapsed, layout_fun, AllenBrain, title=title, use_centrality=True, centrality_metric="Participation coefficient", width=1000, isolated_regions=False, use_clustering=True)
# fig = BraiAn.draw_network_plot(fc_pruned, layout_fun, AllenBrain, title=title, use_centrality=False, use_clustering=False, width=1000, isolated_regions=False)
fig.show()
# fig.write_html(f"chord_{fc_pruned.name.lower()}.html")
fig.write_html(f"{fc_collapsed.name.lower()}.html")

In [ ]:
import importlib
__imported_modules = sys.modules.copy()
for module_name, module in __imported_modules.items():
    if not module_name.startswith("BraiAn"):
        continue
    try:
        importlib.reload(module)
    except ModuleNotFoundError:
        continue